# faiss基于llamaindex的嵌入

为了total-llm项目的文件问答做技术验证

## 准备

In [1]:
%%time
%%capture

!pip install llama_index.core
!pip install llama-index-llms-openai-like
!pip install llama-index-embeddings-ollama
!pip install llama-index-vector-stores-faiss
!pip install faiss-gpu

CPU times: user 38.8 ms, sys: 21.3 ms, total: 60 ms
Wall time: 14.9 s


In [2]:
%%time

import logging
import sys

from llama_index.core import VectorStoreIndex, Document, Settings
from llama_index.core import StorageContext
from IPython.display import Markdown, display
from llama_index.embeddings.ollama import OllamaEmbedding

from llama_index.llms.openai_like import OpenAILike

CPU times: user 2.69 s, sys: 470 ms, total: 3.16 s
Wall time: 3.62 s


In [3]:
Settings.chunk_size=128
Settings.chunk_overlap=10

Settings

_Settings(_llm=None, _embed_model=None, _callback_manager=None, _tokenizer=None, _node_parser=SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f133ae43550>, id_func=<function default_id_func at 0x7f13f93709d0>, chunk_size=128, chunk_overlap=10, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?'), _prompt_helper=None, _transformations=None)

In [4]:
%%time

llm = OpenAILike(model="qwen:14b", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True,
                 temperature=0.1
                )

Settings.llm =llm

CPU times: user 124 ms, sys: 1.05 ms, total: 125 ms
Wall time: 147 ms


In [5]:
%%time

# 初始化全局 embedding 模型
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="dztech/bge-large-zh:v1.5",
    # model_name="bge-m3:latest",
    base_url="http://192.168.0.72:11435",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.embed_model = ollama_embedding

CPU times: user 436 ms, sys: 33.9 ms, total: 470 ms
Wall time: 562 ms


In [6]:
%%time

# 加载文件数据
with open('./books/故乡.txt', 'rb') as file:
    byte_data = file.read()

docs = [Document(text=byte_data)]

CPU times: user 783 µs, sys: 0 ns, total: 783 µs
Wall time: 602 µs


## faiss 索引

In [8]:
%%time

import faiss
from llama_index.vector_stores.faiss import FaissVectorStore

# bge large embedding
d = 1024
faiss_index = faiss.IndexFlatIP(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    docs, storage_context=storage_context
)

CPU times: user 258 ms, sys: 19.1 ms, total: 277 ms
Wall time: 1.74 s


## 基本嵌入查询

In [9]:
%%time

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=10
)
response = query_engine.query("闰土是谁？")
response.print_response_stream()

闰土是小说《故乡》中的主角之一，他是鲁迅先生笔下一个朴实、勤劳的农村少年形象。在故事中，闰土与主人公“我”（迅哥儿）有着深厚的童年友谊。
CPU times: user 145 ms, sys: 15.1 ms, total: 160 ms
Wall time: 5.66 s


In [12]:
response.source_nodes[0].score

218.7655792236328

In [13]:
texts = [node.text for node in response.source_nodes]
print('\n------\n'.join(texts))

我这时很兴奋，但不知道怎么说才好，只是说：

　　“阿！闰土哥，——你来了？
------
我家只有一个忙月（我们这里给人做工的分三种：整年给一定人家做工的叫长工；按日给人做工的叫短工；自己也种地，只在过年过节以及收租时候来给一定人家做工的称忙月），忙不过来，他便对父亲说，可以叫他的儿子闰土来管祭器的。
------
我也说不出话。

　　他回过头去说，“水生，给老爷磕头。”便拖出躲在背后的孩子来，这正是一个廿年前的闰土，只是黄瘦些，颈子上没有银圈罢了。
------
你们先前不是哥弟称呼么？还是照旧：迅哥儿。”母亲高兴的说。

　　“阿呀，老太太真是……这成什么规矩。那时是孩子，不懂事……”闰土说着，又叫水生上来打拱，那孩子却害羞，紧紧的只贴在他背后。
------
我看时，不由的非常出惊，慌忙站起身，迎着走去。

　　这来的便是闰土。虽然我一见便知道是闰土，但又不是我这记忆上的闰土了。
------
什么都有：稻鸡，角鸡，鹁鸪，蓝背……”

　　我于是又很盼望下雪。

　　闰土又对我说：

　　“现在太冷，你夏天到我们这里来。我们日里到海边捡贝壳去，红的绿的都有，鬼见怕也有，观音手⑸也有。
------
我的父亲允许了；我也很高兴，因为我早听到闰土这名字，而且知道他和我仿佛年纪，闰月生的，五行缺土⑷，所以他的父亲叫他闰土。他是能装〔弓京〕捉小鸟雀的。
------
“我们沙地里，潮汛要来的时候，就有许多跳鱼儿只是跳，都有青蛙似的两个脚……”

　　阿！闰土的心里有无穷无尽的希奇的事，都是我往常的朋友所不知道的。
------
“这是第五个孩子，没有见过世面，躲躲闪闪……”

　　母亲和宏儿下楼来了，他们大约也听到了声音。

　　“老太太。信是早收到了。我实在喜欢的不得了，知道老爷回来……”闰土说。

　　“阿，你怎的这样客气起来。
------
的刺去，那猹却将身一扭，反从他的胯下逃走了。

　　这少年便是闰土。我认识他时，也不过十多岁，离现在将有三十年了；那时我的父亲还在世，家景也好，我正是一个少爷。那一年，我家是一件大祭祀的值年⑶。


## reranker

In [14]:
%%time

from llama_index.core.postprocessor import SentenceTransformerRerank

reranker = SentenceTransformerRerank(model='/models/bge-reranker-v2-m3', top_n=5)

CPU times: user 1.88 s, sys: 1.06 s, total: 2.94 s
Wall time: 8.88 s


In [15]:
%%time

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    node_postprocessors=[reranker]
)

CPU times: user 175 µs, sys: 0 ns, total: 175 µs
Wall time: 179 µs


In [16]:
%%time

streaming_response = query_engine.query("宏儿是谁")
streaming_response.print_response_stream()
print()

宏儿是鲁迅先生作品《故乡》中的人物，他是鲁迅八岁的侄子。

CPU times: user 1.17 s, sys: 90.3 ms, total: 1.26 s
Wall time: 6.02 s


In [17]:
streaming_response.source_nodes[0].score

0.73501855

In [18]:
%%time

streaming_response = query_engine.query("我和宏儿是啥关系")
streaming_response.print_response_stream()
print()

我和宏儿是亲戚关系，具体可能是长辈与晚辈或者兄弟姐妹之间的关系。

CPU times: user 496 ms, sys: 6.09 ms, total: 502 ms
Wall time: 4.78 s


In [19]:
%%time

streaming_response = query_engine.query("闰土是谁")
streaming_response.print_response_stream()
print()

闰土是小说《故乡》中的主角之一，由鲁迅先生创作。他是一个朴实的农村少年，与“我”（小说中的叙述者）有过深厚的童年友谊。

CPU times: user 506 ms, sys: 3.55 ms, total: 509 ms
Wall time: 2.38 s


In [20]:
%%time

streaming_response = query_engine.query("闰土和水生是啥关系")
streaming_response.print_response_stream()
print()

闰土和水生可能是亲戚关系或者有其他紧密联系，但具体的关系在给定的文本中并未明确说明。

CPU times: user 503 ms, sys: 3.1 ms, total: 506 ms
Wall time: 2.03 s


In [21]:
%%time

streaming_response = query_engine.query("水生和宏儿是啥关系")
streaming_response.print_response_stream()
print()

水生和宏儿是朋友关系。宏儿被母亲鼓励去与水生交往，说明他们之间有互动和友谊。

CPU times: user 514 ms, sys: 1.02 ms, total: 515 ms
Wall time: 2.08 s


In [22]:
%%time

streaming_response = query_engine.query("闰土小时候和成年后有哪些主要变化")
streaming_response.print_response_stream()
print()

闰土小时候与成年后的变化主要体现在以下几个方面：

1. 社会角色转变：小时候，闰土是农村孩子，过着简单的生活。而成年后，他可能需要承担家庭责任和社会职责。

2. 知识视野拓宽：在鲁迅的记忆中，闰土小时候的世界相对狭窄。而成年后，闰土可能会接触到更广阔的知识领域和信息。

3. 心理成熟度提高：随着年龄的增长，闰土的心理承受能力和决策能力会有所提升。他可能需要处理更为复杂的生活问题和社会冲突。

4. 人际关系变化：小时候的闰土与迅哥儿关系亲密。而成年后，闰土可能会面临更多的人际交往，包括家庭、工作和社区等不同层面的关系。

总之，闰土从小时候到成年后的变化是多方面的，涉及社会角色、知识视野、心理成熟度以及人际关系等多个维度。

CPU times: user 732 ms, sys: 14.5 ms, total: 746 ms
Wall time: 7.86 s
